In [1]:
import nibabel as nib
import numpy as np

In [6]:
a_path = "/home/roland/uni/thesisprep/ds001420-download/derivatives/petprep/sub-dasb02/ses-baseline/pet/sub-dasb02_ses-baseline_space-MNI152NLin2009cAsym_desc-preproc_pet.nii.gz"
b_path = "/home/roland/uni/thesisprep/ds001420-download/derivatives/petprep/sub-dasb01/ses-rescan/pet/sub-dasb01_ses-rescan_space-MNI152NLin2009cAsym_desc-preproc_pet.nii.gz"
mask_path = "/home/roland/uni/thesisprep/ds001420-download/derivatives/petprep/sub-dasb01/ses-baseline/pet/sub-dasb01_ses-baseline_space-MNI152NLin2009cAsym_desc-brain_mask.nii.gz"

In [7]:
a = nib.load(a_path)
b = nib.load(b_path)
m = nib.load(mask_path)


In [8]:
A = a.get_fdata(dtype=np.float32)
B = b.get_fdata(dtype=np.float32)
M = m.get_fdata(dtype=np.float32) > 0

In [9]:
assert A.shape == B.shape , (A.shape, B.shape)

assert np.allclose(a.affine, b.affine), "Affines differ, do not do voxel wise compare."

In [10]:
diff = (A - B)
absdiff = np.abs(diff)

In [11]:
print("mean abs diff in mask:", absdiff[M].mean())
print("max abs diff in mask:", absdiff[M].max())
print("p95 abs diff in mask:", np.quantile(absdiff[M], 0.95))

mean abs diff in mask: 10095.168
max abs diff in mask: 940403.5
p95 abs diff in mask: 34344.055


In [12]:
out = nib.Nifti1Image(absdiff.astype(np.float32), a.affine, a.header)
nib.save(out, "absdiff_MNI_preproc_pet.nii.gz")
